In [ ]:
import requests
import json
import pandas as pd
import os
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

In [ ]:
# This code line allows to remove warning produce by verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
# Define the export function
def generatecsvFiles(export_df,variable):
    for estaciones_id in export_df['id_estacion'].unique():
        data = export_df.query("id_estacion == @estaciones_id")
        data[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/{estaciones_id}_{variable}.csv',float_format='%.3f',index=False)

In [ ]:
archivo = 'run_info.xml'
if os.path.exists(archivo):
    with open(archivo) as f:
        root = ET.parse(f)
        for child in root.iter('*'):
            if child.tag == '{http://www.wldelft.nl/fews/PI}startDateTime':
                fecha = child.attrib['date'] + ' ' + child.attrib['time']
                break
        fin = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        inicio = fin - timedelta(days = 3)
else:
    # Aquí se pueden ingresar un período de fechas para la consulta de niveles
    # Si se deja vacío, la consulta será a las últimas 24 horas
    ###################################
    # PERÍODO (máximo 15 días) #
    # Formato yyyy-mm-dd hh:mm #
    ###################################
    # inicio = '2025-01-01 00:00:00' #
    # fin = '2025-07-23 00:00:00' #
    inicio = ''
    fin = ''
    ###################################
    # Si inicio o fin se dejaron vacías, se haya la fecha actual (fin) y un día antes (inicio)
    if fin == '':
        fin = datetime.today().replace(second=0, microsecond=0)
    else:
        fin = datetime.strptime(fin, '%Y-%m-%d %H:%M:%S')
    if inicio == '':
        inicio = fin - timedelta(days=3)
    else:
        inicio = datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S')
    if inicio > fin:
        raise ValueError("La fecha inicio debe ser inferior a la fecha final")


print(inicio, fin)

In [ ]:
# read api url and credentials from .env
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Dinagua Web Service
url_base = os.environ["url"] # 'https://www.ambiente.gub.uy/dinaguaws/'

In [ ]:
# Username and password
user_dinagua = {"user": os.environ["user"], "password": os.environ["password"]}

In [ ]:
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_dinagua,verify=False);
response_token.close()
# Extract token and assign it to a variable
token = response_token.text
# Add the token to a bearer authorization
headers = {"Authorization" : "Bearer " + token}

get dinagua stations

In [ ]:
# Get Dinagua stations
response_estaciones = requests.get(url=url_base + "service/estaciones",json=user_dinagua, headers = headers, verify=False)
response_estaciones.close()

In [ ]:
# Convert it to json and dataframe variables
estaciones_dinagua = json.loads(response_estaciones.content.decode('utf-8'))
estaciones_dinagua = pd.DataFrame(estaciones_dinagua)

In [ ]:
# export csv
estaciones_dinagua.to_csv("output/estaciones.csv", index=False)

In [ ]:
# Filter station by Activa
estaciones_activa = estaciones_dinagua.query("estado == 'Activa'")
# Get id stations
id_estaciones_activa = estaciones_activa["id_estacion"]
# Get name of the station
nombre_estaciones_activa = estaciones_activa["apodo"]

In [ ]:
display(estaciones_dinagua.query("id_estacion == 51"))

In [ ]:
id_estaciones_activa.to_clipboard()

In [ ]:
# Define the parameters for the get niveles end-point
headers_niveles = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin" : fin.strftime('%Y-%m-%d %H:%M'), "variable" : "H"}
headers_caudales = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin" : fin.strftime('%Y-%m-%d %H:%M'), "variable" : "Q"}
headers_temperatura = {"inicio" : inicio.strftime('%Y-%m-%d %H:%M'), "fin": fin.strftime('%Y-%m-%d %H:%M'), "variable" : "T"}

In [ ]:
# Get niveles from Dinagua WS
response_niveles = requests.get(url= url_base + "service/datos/horarios", params=headers_niveles, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_niveles.close()
# Convert it to json and dataframe
response_niveles = json.loads(response_niveles.content.decode('utf-8'))
response_niveles = pd.DataFrame(response_niveles)

In [ ]:
response_niveles.query("id_estacion == 1330")

In [ ]:
# Get Caudales from Dinagua WS
response_caudales = requests.get(url= url_base + "service/datos/horarios", params=headers_caudales, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_caudales.close()
# Convert it to json and dataframe
response_caudales = json.loads(response_caudales.content.decode('utf-8'))
response_caudales = pd.DataFrame(response_caudales)

In [ ]:
# Get Temperatura from Dinagua WS
response_temperature = requests.get(url= url_base + "service/datos/horarios", params=headers_temperatura, json=user_dinagua, headers = headers, verify=False)
# Close the connection
response_temperature.close()
# Convert it to json and dataframe
response_temperature = json.loads(response_temperature.content.decode('utf-8'))
response_temperature = pd.DataFrame(response_temperature)

In [ ]:
display(response_temperature)

In [ ]:
# Get Daily Discharge
headers_diarios = {
    "inicio" : inicio.strftime('%Y-%m-%d %H:%M'), 
    "fin" : fin.strftime('%Y-%m-%d %H:%M'), 
    "variable" : "Q",
    "tipo_calculo":"Promedio",
    "id_estacion": 1330
}
response_daily = requests.get(
    url= url_base + "service/datos/diarios", 
    params=headers_diarios, 
    json=user_dinagua, 
    headers = headers, 
    verify=False)
# Close the connection
response_daily.close()
# Convert it to json and dataframe
response_daily = json.loads(response_daily.content.decode('utf-8'))
response_daily = pd.DataFrame(response_daily)

In [ ]:
response_daily

In [ ]:
response_niveles[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/niveles.csv',float_format='%.3f',index=False)
response_caudales[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/caudales.csv',float_format='%.3f',index=False)
response_temperature[['id_estacion','fecha','indice_calidad','indice_revision','tipo_calculo','valor']].to_csv(f'output/temperatura.csv',float_format='%.3f',index=False)

## Hydrosos categorized status product

In [ ]:

user_hydrosos = {"user": os.environ["user_hydrosos"], "password": os.environ["password_hydrosos"]}
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_hydrosos,verify=False)
response_token.close()
# Extract token and assign it to a variable
token = response_token.text
# Add the token to a bearer authorization
headers_hydrosos = {"Authorization" : "Bearer " + token}

In [ ]:
# get status product
response_datoscat = requests.get(
    url= url_base + "estadohidro/datoscat", 
    params={
        "serietemporal": "Estaciones", 
        "fecha": "2025-06",
        "temporalidad": "1"}, 
    json=user_hydrosos, 
    headers = headers_hydrosos, 
    verify=False
)
# Close the connection
response_datoscat.close()
# Convert it to json and dataframe
datoscat = json.loads(response_datoscat.content.decode('utf-8'))
df_datoscat = pd.DataFrame(datoscat)

In [ ]:
df_datoscat

In [ ]:
from dateutil.relativedelta import relativedelta
# retrieve recursively for a time period
start_date = datetime(2023, 8, 1)
end_date = datetime(2025, 7, 1)
current_date = start_date
results = []
while current_date < end_date:
    fecha = "%04d-%02d" % (current_date.year, current_date.month)
    # print(fecha)
    response = requests.get(
        url= url_base + "estadohidro/datoscat", 
        params={
            "serietemporal": "Estaciones", 
            "fecha": fecha,
            "temporalidad": "1"}, 
        json=user_hydrosos, 
        headers = headers, 
        verify=False
    )
    # Close the connection
    response.close()
    # print(response)
    # print(response.text)
    if response.status_code != 200:
        current_date = current_date + relativedelta(months=1)
        continue
    # Convert it to json and dataframe
    datoscat = json.loads(response.content.decode('utf-8'))
    df_datoscat = pd.DataFrame(datoscat)
    df_datoscat["fecha"] = current_date
    results.append(df_datoscat)
    current_date = current_date + relativedelta(months=1)
df_datoscat_timeseries = pd.concat(results)

In [ ]:
df_datoscat_timeseries.query("stationID == '1330'")